In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

import matplotlib as mpl
import numpy as np
import pandas as pd

from rcpl.config import BASE_DIR, MPL_RC, CONFIGS_DIR
from rcpl.prediction import ChainPredictor, format_float

mpl.rcParams.update(MPL_RC)
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")

In [3]:
exp_path, e_id = BASE_DIR / 'data/epsp_stress/measured/2023-11-23.json', 1

In [4]:
gru_predictor = ChainPredictor(config_path=CONFIGS_DIR / 'maftr/maftr2_4.best/gru.yaml', predicts_scaled_theta=True)

In [5]:
unscaled_theta_opt_gru, stress_pred_opt_gru, _ = gru_predictor.predict_from_json_simplex(exp_path, verbose=True)

Origin score: 812.12, Nelder-Mead score: 393.02


In [6]:
fraction = 1
validate_crlb_opt = gru_predictor.chain.validate_crlb.force().value(exp_path, crop_signal=[0, 681], unscaled_theta=unscaled_theta_opt_gru, fraction=fraction)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 255.15it/s]


In [7]:
unscaled_theta_prediction_dict = {f'${key}$': val for key, val in zip(gru_predictor.chain.dataset_info.value['latex_labels'], unscaled_theta_opt_gru)}

In [8]:
df_params = pd.DataFrame(unscaled_theta_prediction_dict.items(), columns=['parameter', r'$\widehat{\boldsymbol\theta}$'])
df_params['ESTD'] = [i / fraction for i in validate_crlb_opt['theta_std']]
df_params[r'$\sqrt{\mathrm{CRLB}}$'] = [np.sqrt(i) for i in validate_crlb_opt['crlb']['std']]
df_params[r'ESTD/$\sqrt{\mathrm{CRLB}}$'] = df_params['ESTD'] / df_params[r'$\sqrt{\mathrm{CRLB}}$']
df_params

,parameter,$\widehat{\boldsymbol\theta}$,ESTD,$\sqrt{\mathrm{CRLB}}$,ESTD/$\sqrt{\mathrm{CRLB}}$
0,$k_0$,206.381744,2.606257,0.767659,3.395073
1,$\kappa_1$,34521.804688,138.226364,17.114615,8.076510
2,$\kappa_2$,0.021214,0.000937,0.009655,0.097053
3,$c_1$,19568.664062,349.847931,11.590568,30.183847
4,$c_2$,2186.573242,134.691833,3.118703,43.188417
5,$c_3$,319.294556,64.890541,6.400525,10.138315
6,$c_4$,515.419739,101.086929,8.301653,12.176723
7,$a_1$,68.740921,5.687988,0.468396,12.143534
8,$a_2$,133.712646,5.492246,0.568652,9.658356
9,$a_3$,11.364692,8.813612,1.166787,7.553744
